In [ ]:
import torch
from torch.utils.data import DataLoader, ConcatDataset
import torch.nn as nn 
import os
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt 
from utils.timeseriesdataset import TimeSeriesDataset
from utils.pad_batch import pad_batch, LABEL_PADDING_VALUE
from models.RegressionModel import RegressionModel
import pickle 
from pathlib import Path
from utils.load_data import load_data

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 35
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 2e-6

torch.cuda.empty_cache()
print('The model is running on:', DEVICE) 


# Create DataLoaders

In [ ]:
train_instances = []
val_instances = []
test_instances = []

train_files = list(Path("../data/simulated_tracks").glob("*/train_instances.pkl"))
val_files = list(Path("../data/simulated_tracks").glob("*/val_instances.pkl"))
test_files = list(Path("../data/simulated_tracks").glob("*/test_instances.pkl"))

for file in train_files:
    with open(file, "rb") as f:
        train_instances += pickle.load(f)

for file in val_files:
    with open(file, "rb") as f:
        val_instances += pickle.load(f)

for file in test_files:
    with open(file, "rb") as f:
        test_instances += pickle.load(f)

print("Train data: ", len(train_instances), "Test data: ", len(test_instances), "Val data: ", len(val_instances))

In [ ]:
conc_train = ConcatDataset(train_instances)
conc_val = ConcatDataset(val_instances)
conc_test = ConcatDataset(test_instances)

train_loader = DataLoader(conc_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)
test_loader = DataLoader(conc_test, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)
val_loader = DataLoader(conc_val, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)

print("DataLoader Sizes:", len(train_loader), len(test_loader), len(val_loader))

# DATA_PATH = "../data/simulated_tracks"
# filepaths = list(Path(DATA_PATH).rglob('*.parquet'))
# random.shuffle(filepaths)

# print("Number of files found:", len(filepaths))

# train_data = []
# test_data = []
# val_data = []

# train_data = [TimeSeriesDataset(filepath, augment=True) for filepath in filepaths[:int(len(filepaths)*0.7)]]
# test_data = [TimeSeriesDataset(filepath, augment=False) for filepath in filepaths[int(len(filepaths)*0.7):int(len(filepaths)*0.85)]]
# val_data = [TimeSeriesDataset(filepath, augment=False) for filepath in filepaths[int(len(filepaths)*0.85):]]

In [ ]:
train_data, val_data = load_data() 

print("Train data: ", len(train_data))
print("Val data: ", len(val_data))

conc_train = ConcatDataset(train_data)
conc_val = ConcatDataset(val_data)

training_loader = DataLoader(conc_train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)
val_loader = DataLoader(conc_val, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)

print("Data", len(training_loader), len(val_loader))

# Model
Load the model, optimizer, scheduler, loss


In [ ]:
model = RegressionModel().to(DEVICE)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('models/checkpoints/alpha_runs/runs_{}'.format(timestamp))
model_directory = os.path.join('models/checkpoints/alpha_model', 'model_{}'.format(timestamp))

print(summary(model, input_size=(BATCH_SIZE, 200, 10)))

continuous_loss_fn = nn.L1Loss(reduction='none')
best_val_loss = float("inf")

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2e-6)

scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=3)

# Training Functions

In [8]:
def train_one_epoch(model, optimizer, dataloader):
    model.train()
    running_loss = 0
    runs = 0

    for inputs, alpha_labels,_,_ in dataloader:

        inputs, alpha_labels = inputs.to(DEVICE), alpha_labels.to(DEVICE)
        mask = (alpha_labels != LABEL_PADDING_VALUE).float()

        outputs = model(inputs)
        outputs = outputs.squeeze(-1)
        total_loss = (continuous_loss_fn(outputs, alpha_labels) * mask).sum() / mask.sum()
                
        optimizer.zero_grad()
        total_loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        
        running_loss += total_loss.item()
        runs += 1

        progress_bar.update()

    return running_loss/runs

def evaluate_model(model, dataloader):
    model.eval()
    
    running_val_total = 0.0
    val_runs = 0

    with torch.no_grad():
        for inputs, alpha_labels,_,_ in dataloader:
            
            inputs, alpha_labels = inputs.to(DEVICE), alpha_labels.to(DEVICE)
            mask = (alpha_labels != LABEL_PADDING_VALUE).float()
            
            outputs = model(inputs)  
            outputs = outputs.squeeze(-1)
            total_loss = (continuous_loss_fn(outputs, alpha_labels) * mask).sum() / mask.sum()            
            running_val_total += total_loss.item()
            val_runs += 1
    
    return running_val_total / val_runs

# Train

In [ ]:
os.makedirs(model_directory, exist_ok=True)
# model.load_state_dict(torch.load('/home/haidiri/Desktop/AnDiChallenge2024/models/checkpoints/alpha_model/model_20241025_132405/model_2'))

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    progress_bar = tqdm(total=len(training_loader), desc='Training', position=0)

    avg_training_loss = train_one_epoch(model, optimizer, training_loader)
    val_total_loss  = evaluate_model(model, val_loader)
    
    print(f'Training LOSS: Alpha {avg_training_loss}\n'
          f'Validation LOSS: Alpha {val_total_loss} \n')
    
    writer.add_scalars('Losses', {
        'Training Alpha Loss': avg_training_loss,
        'Validation Alpha Loss': val_total_loss,
        }, epoch + 1)

    writer.flush()
    
    if val_total_loss < best_val_loss:
        best_val_loss = val_total_loss
        best_model_path = os.path.join(model_directory, f'model_{epoch + 1}')
        torch.save(model.state_dict(), best_model_path)

    scheduler.step(val_total_loss)
    
progress_bar.close()
writer.close()

In [ ]:
print("Best Validation Loss:", best_val_loss)
print("Best Model Path", best_model_path)
#  models/checkpoints/alpha_model/model_20241027_120020/model_17

In [ ]:
# model = RegressionModel().to(DEVICE)
# # model.load_state_dict(torch.load('/home/haidiri/Desktop/AnDiChallenge2024/models/optimal_weights/alpha_weights'))
# model.load_state_dict(torch.load('/home/haidiri/Desktop/AnDiChallenge2024/models/checkpoints/alpha_model/model_20241026_150827/model_9'))
# # models/checkpoints/alpha_model/model_20241026_150827/model_9
# model.eval()

# running_test_total = 0.0
# test_runs = 0.0

# predictions = []
# ground_truth = []

# progress_bar = tqdm(total=len(val_loader), desc='Testing', position=0)

# with torch.no_grad():
#     for inputs, alpha_labels,_,_ in val_loader:
        
#         inputs, alpha_labels = inputs.to(DEVICE), alpha_labels.to(DEVICE)

#         mask = (alpha_labels != LABEL_PADDING_VALUE).float()
#         outputs = model(inputs).squeeze(-1)
#         total_loss = (continuous_loss_fn(outputs, alpha_labels) * mask).sum() / mask.sum()
        
#         running_test_total += total_loss.item()
#         test_runs += 1

#         predictions.extend(outputs.cpu().numpy())
#         ground_truth.extend(alpha_labels.cpu().numpy())
#         progress_bar.update()


# # Calculate average losses
# avg_test_loss = running_test_total / test_runs
# print(f'Average test loss: {avg_test_loss}')
# progress_bar.close()

# Testing: 100%|██████████| 57094/57094 [13:21<00:00, 71.25it/s]
# non finetuned model
# Average test loss: 0.11597597394306262

# Testing

In [14]:
with open("/home/haidiri/Desktop/AnDiChallenge2024/data/simulated_tracks/simulations_general/test_instances.pkl", "rb") as file:
    test_data = pickle.load(file)

with open("/home/haidiri/Desktop/AnDiChallenge2024/data/simulated_tracks/simulations_3bias/test_instances.pkl", "rb") as file:
    test_data.extend(pickle.load(file))

with open("/home/haidiri/Desktop/AnDiChallenge2024/data/simulated_tracks/simulations_default/test_instances.pkl", "rb") as file:
    test_data.extend(pickle.load(file))

# with open("/home/haidiri/Desktop/AnDiChallenge2024/data/simulated_tracks/simulation_alpha_K_one_fixed/test_instances.pkl", "rb") as file:
#     test_data.extend(pickle.load(file))

conc_test = ConcatDataset(test_data)
test_loader = DataLoader(conc_test, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_batch)

In [ ]:
model = RegressionModel().to(DEVICE)
model.load_state_dict(torch.load('/home/haidiri/Desktop/AnDiChallenge2024/models/optimal_weights/alpha_weights_with_fixed'))
# model.load_state_dict(torch.load('/home/haidiri/Desktop/AnDiChallenge2024/src/models/checkpoints/alpha_model/model_20241026_150827/model_9'))
# models/checkpoints/alpha_model/model_20241026_150827/model_9
model.eval()

running_test_total = 0.0
test_runs = 0.0

predictions = []
ground_truth = []

progress_bar = tqdm(total=len(test_loader), desc='Testing', position=0)

with torch.no_grad():
    for inputs, alpha_labels,_,_ in test_loader:
        
        inputs, alpha_labels = inputs.to(DEVICE), alpha_labels.to(DEVICE)

        mask = (alpha_labels != LABEL_PADDING_VALUE).float()
        outputs = model(inputs).squeeze(-1)
        total_loss = (continuous_loss_fn(outputs, alpha_labels) * mask).sum() / mask.sum()
        
        running_test_total += total_loss.item()
        test_runs += 1

        predictions.extend(outputs.cpu().numpy())
        ground_truth.extend(alpha_labels.cpu().numpy())
        progress_bar.update()


# Calculate average losses
avg_test_loss = running_test_total / test_runs
print(f'Average test loss: {avg_test_loss}')
progress_bar.close()

# Average test loss: 0.11042290411644512 with normal weights
# Average test loss: 0.11053658076882363 with alpha fixed


# Plot Predictions

In [ ]:
INDEX = 0
import numpy as np
padding_starts = (ground_truth[INDEX] == LABEL_PADDING_VALUE).argmax() 

if padding_starts == 0:
    padding_starts = 200

pred_alpha = predictions[INDEX][:padding_starts]
true_alpha = ground_truth[INDEX][:padding_starts]
print(np.abs(pred_alpha - true_alpha).mean())   

plt.scatter([i for i in range(len(pred_alpha))], pred_alpha, color="red")
plt.scatter([i for i in range(len(true_alpha))], true_alpha, color="blue")
plt.title("Alpha")
plt.show()